In [9]:
import cv2
import face_recognition
import numpy as np
import os
import matplotlib.pyplot as plt

**Face Recognition**

In [24]:
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

Create Encoding of Known Faces

In [25]:
known_face_encodings = []
known_face_names = []

path = "known_faces"
for filename in os.listdir(path):
    img_path = os.path.join(path, filename)
    name = os.path.splitext(filename)[0]
    img = face_recognition.load_image_file(img_path)
    encoding = face_recognition.face_encodings(img)
    if encoding:
        known_face_encodings.append(encoding[0])
        known_face_names.append(name)

Haar Face Detection

In [26]:
def detect_faces(img):
    face_img = img.copy()
    face_rects = face_cascade.detectMultiScale(face_img, scaleFactor=1.2, minNeighbors=5)
    return face_rects

face_recognition Library for Recognizing New Faces by Comparing with Known Faces

In [27]:
def recognize_faces(img, face_rects):
    face_img = img.copy()
    for (x, y, w, h) in face_rects:
        roi = face_img[y:y+h, x:x+w]
        rgb_face = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        encodings = face_recognition.face_encodings(rgb_face)
        name = "Unknown"
        if encodings:
            matches = face_recognition.compare_faces(known_face_encodings, encodings[0])
            face_distances = face_recognition.face_distance(known_face_encodings, encodings[0])

            if True in matches:
                best_match_index = np.argmin(face_distances)
                name = known_face_names[best_match_index]
        
        cv2.rectangle(face_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(face_img, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
    return face_img

Main Real-Time Face Recognition

In [28]:
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    if not ret:
        break
    
    face_rects = detect_faces(frame)
    result = recognize_faces(frame, face_rects)
    cv2.imshow("Face Recognition", result)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cam.release()
cv2.destroyAllWindows()


**Image Quality Assessment**

In [16]:
path = "IQA"

Detect Blur

In [29]:
def is_blur(img):
    threshold=100
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    variance = cv2.Laplacian(gray, cv2.CV_64F).var()
    return variance < threshold, variance

In [30]:
detect_blur_path = "IQA/detect_blur"
for filename in os.listdir(detect_blur_path):
    img_path = os.path.join(detect_blur_path, filename)
    img = cv2.imread(img_path)
    text = "Not Blurry"
    blurred, variance = is_blur(img)
    if blurred:
        text = "Blurry"
    
    cv2.putText(img, f"{text}: {variance:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Image Quality Assessment - Blur", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Detect Exposure

In [18]:
def classify_exposure(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    total_pixels = gray.size
    dark_pixels = np.sum(hist[:50])
    bright_pixels = np.sum(hist[205:])

    dark_ratio = dark_pixels / total_pixels
    bright_ratio = bright_pixels / total_pixels

    if dark_ratio > 0.3:
        return "Underexposed"
    elif bright_ratio > 0.3:
        return "Overexposed"
    else:
        return "Well-exposed"

In [ ]:
detect_exposure_path = "IQA/detect_exposure"

for filename in os.listdir(detect_exposure_path):
    img_path = os.path.join(detect_exposure_path, filename)
    img = cv2.imread(img_path)
    
    exposure = classify_exposure(img)

    cv2.putText(img, f"{exposure}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Image Quality Assessment - Exposure", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Detect Contrast

In [ ]:
%pip install scikit-image

In [19]:
from skimage.exposure import is_low_contrast

detect_contrast_path = "IQA/detect_contrast"
for filename in os.listdir(detect_contrast_path):
    img_path = os.path.join(detect_contrast_path, filename)
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    if is_low_contrast(gray, 0.35):
        text = "Low Contrast"
    elif not is_low_contrast(gray, 0.85):
        text = "High Contrast"
    else:
        text = "Normal Contrast"
    
    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Image Quality Assessment - Contrast", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Detect Noise

In [20]:
def detect_noise(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 0)
    diff = cv2.absdiff(gray, blurred)

    mean, std = np.mean(diff), np.std(diff)

    if mean > 10 or std > 20:
        return True
    return False

In [42]:
detect_noise_path = "IQA/detect_noise"

for filename in os.listdir(detect_noise_path):
    img_path = os.path.join(detect_noise_path, filename)
    img = cv2.imread(img_path)
    noise_detected = detect_noise(img)

    if noise_detected:
        text = "Noisy"
    else:
        text = "Not Noisy"
    
    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Image Quality Assessment - Noise", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

**Annotation Capability**

Drawing Rectangle

In [31]:
def draw_rectangle(event, x, y, flags, param):
    global drawing, start_x, start_y

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_x, start_y = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            cv2.rectangle(img_copy, (start_x, start_y), (x, y), (0, 255, 0), 2)
            cv2.imshow("image", img_copy)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, (start_x, start_y), (x, y), (0, 255, 0), 2)
        cv2.imshow("image", img)

Drawing Circle

In [32]:
def draw_circle(event, x, y, flags, param):
    global drawing, start_x, start_y

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_x, start_y = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            center_x = (start_x + x) // 2
            center_y = (start_y + y) // 2
            radius = int(0.5 * np.sqrt((x - start_x)**2 + (y - start_y)**2))
            cv2.circle(img_copy, (center_x, center_y), radius, (0, 0, 255), 2)
            cv2.imshow("image", img_copy)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        center_x = (start_x + x) // 2
        center_y = (start_y + y) // 2
        radius = int(0.5 * np.sqrt((x - start_x)**2 + (y - start_y)**2))
        cv2.circle(img, (center_x, center_y), radius, (0, 0, 255), 2)
        cv2.imshow("image", img)

Drawing Line

In [33]:
def draw_line(event, x, y, flags, parmas):
    global drawing, start_x, start_y

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_x, start_y = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            cv2.line(img_copy, (start_x, start_y), (x, y), (255, 0, 0), 2)
            cv2.imshow("image", img_copy)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.line(img, (start_x, start_y), (x, y), (255, 0, 0), 2)
        cv2.imshow("image", img)

Writing Text

In [34]:
def write_text(event, x, y, flags, param):
    global img
    if event == cv2.EVENT_LBUTTONDOWN:
        text = input("Enter text to write: ")
        cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("image", img)

Main Annotation Logic

In [35]:
img = cv2.imread("known_faces/elon musk.jpg")
shape = int(input("Enter shape to draw: \n1. Circle \n2. Rectangle \n3. Line, \n4. Text \n5. Exit): "))

drawing = False
start_x, start_y = -1, -1

if shape == 1:
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", draw_circle)
elif shape == 2:
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", draw_rectangle)
elif shape == 3:
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", draw_line)
elif shape == 4:
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", write_text)
else:
    print("Exiting...")
    

while True:
    cv2.imshow("image", img)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()

In [8]:
%pip install streamlit

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import streamlit as st

st.title("🧠 Face Recognition + IQA + Drawing Tool")

option = st.sidebar.selectbox("Select Functionality", ["Face Recognition", "Image Quality Assessment", "Draw on Image"])

if option == "Face Recognition":
    uploaded_file = st.file_uploader("Upload Image", type=["jpg", "png"])
    if uploaded_file:
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        img = cv2.imdecode(file_bytes, 1)
        result = recognize_faces(img.copy())
        st.image(cv2.cvtColor(result, cv2.COLOR_BGR2RGB), caption="Processed Image", use_column_width=True)

elif option == "Image Quality Assessment":
    uploaded_file = st.file_uploader("Upload Image", type=["jpg", "png"])
    if uploaded_file:
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        img = cv2.imdecode(file_bytes, 1)
        blurred, var = is_blur(img)
        exposure = classify_exposure(img)
        contrast = detect_contrast(img)
        noisy = detect_noise(img)

        st.image(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), caption="Uploaded Image", use_column_width=True)
        st.markdown(f"- **Blur**: {'Blurry' if blurred else 'Not Blurry'} (Variance: {var:.2f})")
        st.markdown(f"- **Exposure**: {exposure}")
        st.markdown(f"- **Contrast**: {contrast}")
        st.markdown(f"- **Noise**: {'Noisy' if noisy else 'Not Noisy'}")

elif option == "Draw on Image":
    uploaded_file = st.file_uploader("Upload Image", type=["jpg", "png"])
    draw_shape = st.selectbox("Shape", ["Rectangle", "Circle", "Line", "Text"])
    if uploaded_file:
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        img = cv2.imdecode(file_bytes, 1)
        img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        st.image(img_pil, caption="Image to Draw", use_column_width=True)
        st.info("Download the image and use OpenCV GUI for interactive drawing (Streamlit doesn't support drawing canvas natively).")